# CodeX

## Eval

生成 $n > k$ 个答案，若其中有 $c$ 个答案能通过单元测试，则

$$
\text{pass}@k := \mathbb{E}_{\text{Problems}}\left[1 - \frac{\binom{n-c}{k} }{\binom{n}{k} }\right]
$$

即选 $k$ 个答案其中至少有一个正确的概率。

不能使用 BLEU score（比较子序列的重合度）。

## 在代码上的微调

使用 GPT3 模型，以 Github 公开的 Python 文件为数据集进行训练，会对空格回车等做特殊处理。

这里我们从 GPT3 的参数开始微调，微调相比直接训练不会提升精度，但会加快收敛。

训练器为 Adam。

## Predict

生成答案时什么时候停？when encounter: ‘nnclass’, ‘nndef’, ‘nn#’, ‘nnif’, or ‘nnprint’.

采样方法：核采样，非 beam search，保留的词的概率 >= 95%

温度 matters

对答案进行排序：选概率均值最高的效果会比较好

## 有监督的微调

Github 上的训练数据集和 HumanEval 测试集很不一样。

所以做了一个额外的训练集，里面的数据格式和 HumanEval 类似，可以提升性能。

使用代码竞赛题目等，再过最大 CodeX 模型，如果100次至少有1次生成正确答案，就把问题留下。（说明太难或单元测试不正确）

## 使用代码生成 docstring

Python 里：函数签名 -> docstring -> 函数实现

所以为了完成这个任务，所以重新做了一个训练数据集，把顺序改成：函数签名 -> 函数实现 -> docstring，计算损失时只看 docstring 里的损失。

评估不能用 HumanVal，而是用眼睛看对不对~ 还有一种方法就是用函数签名 + 生成的docstring 生成代码，看代码能不能过单元测试。